# ChessVision Performance Report

This notebook can be used to generate a full performance report of the ChessVision system.

### Imports

In [ ]:
%matplotlib inline

from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
import chessvision as cv
from test import run_tests, get_test_generator
import cv_globals
import cv2
import numpy as np
from model.u_net import load_extractor
from model.square_classifier import load_classifier

### Parameters

These variables will be filled in by papermill. 
(Default to using current best models)

In [ ]:
classifier_weights = cv_globals.square_weights
extractor_weights  = cv_globals.board_weights
threshold          = 80

### Load models and test data generator

In [ ]:
data_generator = get_test_generator()
extractor      = load_extractor(weights=extractor_weights)
classifier     = load_classifier(weights=classifier_weights)

In [ ]:
results = run_tests(data_generator, extractor, classifier, threshold)

### Get results

In [ ]:
board_imgs = results["board_imgs"]
imgs = results["raw_imgs"]
predictions = results["predictions"]
boards = results["chessboards"]
squares = results["squares"]
acc = results["acc"]
avg_time = results["avg_time"]
squares = results["squares"]
N = len(results["raw_imgs"])

In [ ]:
def entropy(dist):
    return sum([-p * np.log(p) for p in dist if p != 0])

def avg_entropy(predictions):
    return sum([entropy(p) for p in predictions]) / len(predictions)

In [ ]:
print("Test suite accuracy: {:.1f}%".format(acc*100))
print("Average time per raw classification: {:.1f}s".format(avg_time))

clf_entropy = sum([avg_entropy(p) for p in predictions]) / len(predictions)

print("Average entropy classifier predictions: {:.2g}".format(clf_entropy))

### Plot results

In [ ]:
rows = N
cols = 2

fig = plt.figure(1, figsize=(12, 40)) #width, height in inches.

grid = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(rows, cols),  # creates 2x2 grid of axes
                 axes_pad=0.1,  # pad between axes in inch.
                 share_all=True,
                 label_mode=None
                 )

for i in range(rows):
    raw = imgs[i]
    board = board_imgs[i]
    
    ax1 = grid[cols*i]
    ax1.imshow(raw)
    ax1.axis("off")
    
    ax2 = grid[cols*i+1]
    ax2.imshow(board, cmap="gray")
    ax2.axis("off")
    
    if i == 0:
        ax1.set_title("Raw")
        ax2.set_title("Extracted board")

#plt.savefig("../../img/training_extraction.png", bbox_inches="tight")
plt.show()


In [ ]:

label_names  = ['B', 'K', 'N', 'P', 'Q', 'R', 'b', 'k', 'n', 'p', 'q', 'r', 'f']

for k in range(N):
    prediction = np.argmax(predictions[k], axis=1)
    pred_labels = [label_names[p] for p in prediction]
    square_grid = squares[k]
    # Plot
    rows, cols = 8, 8
    fig = plt.figure(figsize=(12, 12))
    grid = ImageGrid(fig, 111,  # similar to subplot(111)
                     nrows_ncols=(rows, cols),  # creates 2x2 grid of axes
                     axes_pad=0.3,  # pad between axes in inch.
                     share_all=True,
                     label_mode=None
                     )

    for i in range(cols):
        for j in range(rows):
            ind = cols*i+j
            im = square_grid[ind]
            label = pred_labels[ind]
            grid[ind].imshow(im.reshape(64, 64), cmap="gray")
            grid[ind].set_title(label, size=14, fontweight=3)
            grid[ind].axis("off")

    plt.suptitle("Classification of test image {}".format(k), size=20, fontweight=5)